In [20]:
import os
import rasterio as rio
from rasterio.plot import show
from rasterio.features import shapes

import geopandas as gpd
from shapely.geometry import Polygon

In [22]:
rloc = '/home/jovyan/data-store/data/iplant/home/shared/earthlab/forest_carbon_codefest/Disturbance/simple_dist_stack_Southern_Rockies.tif'
nodata_values = [0, 255]


with rio.open(rloc) as r:
    prof = r.profile.copy()
    desc = r.descriptions

out_dir = './disturbance_shps'
if not os.path.exists(out_dir): os.mkdir(out_dir)

for i in range(prof['count']):
    with rio.open(rloc) as r:
        dat = r.read(i+1)
    name = desc[i]
    
    sh = shapes(dat, transform = prof['transform'])
    geom = []
    vals = []
    for s in sh:
        geom.append(Polygon(s[0]['coordinates'][0]))
        vals.append(s[1])
    
    dist_feat = gpd.GeoDataFrame(
        {'value':vals, 'geometry':geom},
        crs=prof['crs'],
    )
    out_shp_dir = os.path.join(out_dir, name)
    
    for ndv in nodata_values:
        dist_feat = dist_feat[dist_feat.value!=ndv].reset_index(drop=True)

    dist_feat.to_file(out_shp_dir)